# Incorporating sensitivy analysis into optimisation

The introductory notebooks show how to perform [optimisation](Optimisation_demo.ipynb) and [sensitivity analysis](Sensitivity_analysis_demo.ipynb) in the PyBiopharma framework. In more advanced cases, you may want to incorporate the results of a sensitivity analysis into the optimisation itself, for example to find configurations which minimise variance in an aspect of interest, while potentially also optimising another aspect.

This notebook demonstrates how to combine the two analysis procedures. It assumes that you have already gone through the previous examples and are familiar with how to create and use the components involved.

The workflow is similar to the simple cases: first we define a facility on which to perform the operations.

In [ ]:
import biopharma as bp

facility = bp.Facility(data_path='data')

# Define the steps needed to create our single product
from biopharma.process_steps import (
    
)
steps = [
    
]
product = bp.Product(facility, steps)
facility.load_parameters()

The next step is to define the sensitivity analysis, which will serve as the basis for evaluating candidate configurations during the optimisation. In other words, for each individual in the population, we will take many samples with varying parameters, and record the results (as in a standard sensitivity analysis). The statistics of these runs will then be used for computing the individual's fitness in the optimisation step.

The definition of the sensitivity analyser is the same as in the [simple demo](Sensitivity_analysis_demo.ipynb). For simplicity, we only choose two output variables to record, and only vary one aspect.

In [ ]:
from biopharma import optimisation as opt

analyser = opt.SensitivityAnalyser(facility)

# Specify the variables whose sensitivity we are interested in, and name them for future reference
analyser.add_output("CoG", component=opt.sel.product(0), item="cogs")
analyser.add_output("step_int_param", component=opt.sel.step('test_step'), item="int_param")

# Specify which aspects to vary
param1_mean = facility.products[0].parameters["param1"]
param1_width = 2 * param1_mean.units
analyser.add_variable(gen=opt.dist.Uniform(param1_mean - param1_width, param1_mean + param1_width),
                      component=opt.sel.product(), item="param")

Following this, we can create the optimiser. In contrast to the usual case, we will use the sensitivity analyser as the base component instead of directly giving the facility. This will allow us to use the outputs of the sensitivity analysis in the optimisation.

In [ ]:
optimiser = opt.Optimiser(analyser)

# Specify the variables to optimise
optimiser.add_variable(gen=opt.gen.Binary(), component=opt.sel.step('test_step'), item='binary_param')
optimiser.add_variable(gen=opt.gen.RangeGenerator(0, 10),
                       component=opt.sel.step('test_step'), item='range_param')

Specifying the optimisation objectives requires a little more care than in the simple case. Now that sensitivity analysis is performed, we no longer have a single value for the aspects we want to optimise; instead, we have a whole set of statistics about them (gathered from all the runs performed). This gives us access to more information, but also means that we cannot refer to the value of a variable of interest directly.

For example, let us assume that we are interested in minimising the cost of goods. To evaluate the fitness of a single set of variables, we instantiate multiple facilities, each parameterised differently. Therefore, we have no way of referring to a single facility, nor to the value of the cost for that particular instance. If we so wish, we can refer to its *average*, which will give an estimate of the cost across the different runs. For this reason, our optimisation objectives must be looked up on the sensitivity analyser rather than the facility.

Note the different way of describing this, compared to the optimisation-only case. Instead of using a selector function for the product, we now use the `opt.sel.self` function (which will select the sensitivity analyser). We also use a "compound" item specification, containing the name of the variable (`"CoG"`) and the statistical measure we want (the choices are `"avg"`, `"var"`, `"min"` and `"max"`, as described in the [sensitivity analysis demo](Sensitivity_analysis_demo.ipynb)).

Note also that the variable of interest must have been defined as an output to the sensitivity analyser! Indeed, we must use the same name to refer to it (`CoG`, in this case).

**TODO**: Should maybe stress that we must have `collections="outputs"`, as the statistics are stored in `analyser.outputs`, but this is the default value for `collection` in `add_objective` so I've omitted it

In [ ]:
optimiser.add_objective(component=opt.sel.self(), item=("CoG", "avg"), minimise=True)

This approach is equally applicable, whether we have a single or [multiple objectives](Multi-objective_optimisation_demo.ipynb). In the same way as above, we can specify that we want to minimise the the variance in the `step_int_param`.

In [ ]:
optimiser.add_objective(component=opt.sel.self(), collection="outputs", item=("step_int_param", "var"), minimise=True)

We are now ready to run the optimisation. Since each fitness evaluation will involve running the facility multiple times, the optimisation run will take significantly longer than without the sensitivity analysis. For this reason, and to limit the execution time of this demo, we lower the size of the population, the number of generations, and the number of samples taken in each sensitivity run.

In [ ]:
# Configured to perform a faster, but less elaborate, search
optimiser.parameters["maxGenerations"] = 5
optimiser.parameters["populationSize"] = 5
analyser.parameters["numberOfSamples"] = 10

# Run the optimisation (including sensitivity analysis at each step)
optimiser.run()

Once the computation is complete, we can access the optimisation results as usual. Each individual in the final population will contain the variables specified for the optimiser.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib notebook

best = optimiser.outputs['bestIndividuals'][0]
# etc.